In [1]:
import pandas as pd

from ddeserts.annotate import add_all_stat_columns
from ddeserts.annotate import with_columns_sorted
from ddeserts.annotate import RACES
from ddeserts.load import load_cvap_data

In [2]:
df = load_cvap_data('State')

In [3]:
# add extra columns
add_all_stat_columns(df)
df = with_columns_sorted(df)
df

geoid               geoname   adu_est  adu_moe   cit_est  \
table line                                                                 
State 1     04000US01               Alabama   3779875      543   4770940   
      14    04000US02                Alaska    552675      202    712455   
      27    04000US04               Arizona   5414955      227   6519390   
      40    04000US05              Arkansas   2295100      584   2903080   
      53    04000US06            California  30261350      462  34187375   
      66    04000US08              Colorado   4349345      496   5298855   
      79    04000US09           Connecticut   2831240      236   3326750   
      92    04000US10              Delaware    753565      118    909575   
      105   04000US11  District of Columbia    568755       91    639585   
      118   04000US12               Florida  16719175      616  18993615   
      131   04000US13               Georgia   7898605     1097   9809250   
      144   04000US15                Hawaii   1117455        0   1313265   
      157   04000US16                 Idaho   1276605      302   1656805   
      170   04000US17              Illinois   9879105      621  11884945   
      183   04000US18               Indiana   5093210      641   6455225   
      196   04000US19                  Iowa   2410770      499   3039680   
      209   04000US20                Kansas   2199580      484   2784630   
      222   04000US21              Kentucky   3439745      877   4345650   
      235   04000US22             Louisiana   3561010      631   4549895   
      248   04000US23                 Maine   1082995      158   1314655   
      261   04000US24              Maryland   4677165      215   5577240   
      274   04000US25         Massachusetts   5479295      231   6314695   
      287   04000US26              Michigan   7787385      344   9639085   
      300   04000US27             Minnesota   4267530      328   5335310   
      313   04000US28           Mississippi   2270925      448   2941850   
      326   04000US29              Missouri   4723300      723   5971680   
      339   04000US30               Montana    822265      404   1040280   
      352   04000US31              Nebraska   1440465      333   1830380   
      365   04000US32                Nevada   2291340      212   2681290   
      378   04000US33         New Hampshire   1088160      153   1311780   
      391   04000US34            New Jersey   6916980      383   8011800   
      404   04000US35            New Mexico   1603980      360   1976245   
      417   04000US36              New York  15463820      352  17686675   
      430   04000US37        North Carolina   7968260      697   9766885   
      443   04000US38          North Dakota    580975      321    738305   
      456   04000US39                  Ohio   9050385      534  11397175   
      469   04000US40              Oklahoma   2975905      425   3782285   
      482   04000US41                Oregon   3261860      334   3905205   
      495   04000US42          Pennsylvania  10129140      403  12379320   
      508   04000US44          Rhode Island    849885       99    994510   
      521   04000US45        South Carolina   3918305      365   4873320   
      534   04000US46          South Dakota    656530      415    850795   
      547   04000US47             Tennessee   5204435      826   6499470   
      560   04000US48                 Texas  20922410     1090  25253945   
      573   04000US49                  Utah   2173265      324   2938730   
      586   04000US50               Vermont    507660      150    610980   
      599   04000US51              Virginia   6588765      957   7953315   
      612   04000US53            Washington   5760560      447   6850930   
      625   04000US54         West Virginia   1447865      579   1802675   
      638   04000US55             Wisconsin   4509815      376   5633105   
      651   04000US56               Wyoming    444655      456  

In [4]:
df.to_csv('output/states.csv')

In [5]:
dg = df[['geoname', 'racial_disp_his_est', 'racial_disp_his_moe', 'prop_adu_dvap_est', 'prop_adu_dvap_moe']]
dg.sort_values(by=['racial_disp_his_est'])

geoname  racial_disp_his_est  racial_disp_his_moe  \
table line                                                                   
State 365                 Nevada            -0.059669             0.001735   
      560                  Texas            -0.059634             0.001063   
      53              California            -0.053909             0.001030   
      118                Florida            -0.042880             0.001071   
      27                 Arizona            -0.042721             0.001280   
      573                   Utah            -0.038229             0.001276   
      170               Illinois            -0.037057             0.000688   
      391             New Jersey            -0.036087             0.001071   
      261               Maryland            -0.033792             0.000851   
      131                Georgia            -0.032816             0.000670   
      430         North Carolina            -0.032330             0.000528   
      66                Colorado            -0.031438             0.001158   
      482                 Oregon            -0.030824             0.001095   
      417               New York            -0.029670             0.000719   
      105   District of Columbia            -0.029240             0.002350   
      612             Washington            -0.028979             0.000738   
      209                 Kansas            -0.028587             0.000883   
      469               Oklahoma            -0.028404             0.000684   
      352               Nebraska            -0.027465             0.001133   
      599               Virginia            -0.027423             0.000816   
      404             New Mexico            -0.026654             0.002329   
      157                  Idaho            -0.025872             0.001241   
      508           Rhode Island            -0.024571             0.001998   
      40                Arkansas            -0.024449             0.000819   
      92                Delaware            -0.022109             0.001349   
      79             Connecticut            -0.020913             0.001108   
      521         South Carolina            -0.019050             0.000638   
      547              Tennessee            -0.018849             0.000464   
      183                Indiana            -0.016607             0.000491   
      274          Massachusetts            -0.016384             0.000692   
      235              Louisiana            -0.015881             0.000644   
      300              Minnesota            -0.015024             0.000493   
      638              Wisconsin            -0.014314             0.000429   
      651                Wyoming            -0.013919             0.001799   
      196                   Iowa            -0.013895             0.000570   
      1                  Alabama            -0.013678             0.000544   
      222               Kentucky            -0.011851             0.000446   
      495           Pennsylvania            -0.009145             0.000361   
      313            Mississippi            -0.009130             0.000791   
      326               Missouri            -0.008473             0.000373   
      287               Michigan            -0.008368             0.000313   
      534           South Dakota            -0.006819             0.000897   
      456                   Ohio            -0.005445             0.000212   
      378          New Hampshire            -0.004754             0.000731   
      14                  Alaska            -0.004281             0.001077   
      443           North Dakota            -0.003219             0.000915   
      339                Montana            -0.002227             0.000583   
      625          West Virginia            -0.001931             0.000571   
      586                Vermont            -0.001822             0.000500   
      248                  Maine            -0.

In [6]:
dg = df[['geoname', 'racial_disp_asn_est', 'racial_disp_asn_moe', 'prop_adu_dvap_est', 'prop_adu_dvap_moe']]
dg.sort_values(by=['racial_disp_asn_est'])

geoname  racial_disp_asn_est  racial_disp_asn_moe  \
table line                                                                   
State 144                 Hawaii            -0.025896             0.003891   
      391             New Jersey            -0.024454             0.000591   
      612             Washington            -0.024329             0.000758   
      417               New York            -0.022318             0.000396   
      274          Massachusetts            -0.021028             0.000583   
      14                  Alaska            -0.018939             0.002000   
      53              California            -0.016378             0.000415   
      599               Virginia            -0.016371             0.000573   
      92                Delaware            -0.015922             0.001250   
      79             Connecticut            -0.015351             0.000654   
      170               Illinois            -0.015237             0.000368   
      261               Maryland            -0.014765             0.000569   
      131                Georgia            -0.014132             0.000434   
      560                  Texas            -0.013056             0.000345   
      287               Michigan            -0.012507             0.000303   
      482                 Oregon            -0.012318             0.000784   
      300              Minnesota            -0.012265             0.000456   
      495           Pennsylvania            -0.011894             0.000277   
      365                 Nevada            -0.011369             0.001131   
      196                   Iowa            -0.011357             0.000534   
      209                 Kansas            -0.011205             0.000685   
      183                Indiana            -0.010981             0.000421   
      378          New Hampshire            -0.010861             0.000885   
      352               Nebraska            -0.010780             0.000742   
      508           Rhode Island            -0.010762             0.001296   
      105   District of Columbia            -0.010606             0.001670   
      430         North Carolina            -0.010057             0.000324   
      27                 Arizona            -0.009940             0.000533   
      456                   Ohio            -0.009712             0.000303   
      66                Colorado            -0.008863             0.000552   
      573                   Utah            -0.008851             0.000727   
      443           North Dakota            -0.008724             0.000956   
      638              Wisconsin            -0.008630             0.000415   
      326               Missouri            -0.007969             0.000339   
      534           South Dakota            -0.007559             0.000942   
      469               Oklahoma            -0.007402             0.000470   
      547              Tennessee            -0.006822             0.000347   
      586                Vermont            -0.006550             0.001277   
      118                Florida            -0.006067             0.000277   
      40                Arkansas            -0.005936             0.000508   
      222               Kentucky            -0.005808             0.000337   
      235              Louisiana            -0.005625             0.000419   
      1                  Alabama            -0.005287             0.000320   
      521         South Carolina            -0.005283             0.000349   
      157                  Idaho            -0.004873             0.000855   
      404             New Mexico            -0.004637             0.000684   
      248                  Maine            -0.003752             0.000584   
      313            Mississippi            -0.003611             0.000476   
      651                Wyoming            -0.003499             0.001071   
      625          West Virginia            -0.

In [7]:
dg = df[['geoname', 'racial_disp_wht_est', 'racial_disp_wht_moe', 'prop_adu_dvap_est', 'prop_adu_dvap_moe']]
dg.sort_values(by=['racial_disp_wht_est'], ascending=False)

geoname  racial_disp_wht_est  racial_disp_wht_moe  \
table line                                                                   
State 365                 Nevada             0.059456             0.001423   
      560                  Texas             0.059330             0.000674   
      53              California             0.058141             0.000624   
      391             New Jersey             0.054490             0.001020   
      612             Washington             0.051834             0.001070   
      417               New York             0.051141             0.000813   
      573                   Utah             0.048131             0.001420   
      118                Florida             0.047215             0.000891   
      27                 Arizona             0.046299             0.000964   
      274          Massachusetts             0.043855             0.001376   
      482                 Oregon             0.042561             0.001329   
      170               Illinois             0.042480             0.000796   
      352               Nebraska             0.040148             0.001439   
      66                Colorado             0.040038             0.001138   
      261               Maryland             0.039401             0.000914   
      508           Rhode Island             0.039132             0.002711   
      79             Connecticut             0.038469             0.001576   
      209                 Kansas             0.038339             0.001132   
      599               Virginia             0.036111             0.000875   
      300              Minnesota             0.033803             0.000883   
      92                Delaware             0.033753             0.001944   
      430         North Carolina             0.032447             0.000590   
      131                Georgia             0.032152             0.000736   
      157                  Idaho             0.030628             0.001656   
      469               Oklahoma             0.029203             0.000737   
      196                   Iowa             0.028955             0.001122   
      183                Indiana             0.026877             0.000741   
      40                Arkansas             0.026698             0.000935   
      404             New Mexico             0.024660             0.001232   
      638              Wisconsin             0.022231             0.000720   
      547              Tennessee             0.022002             0.000749   
      495           Pennsylvania             0.021844             0.000531   
      534           South Dakota             0.018485             0.001661   
      222               Kentucky             0.018431             0.000829   
      14                  Alaska             0.018357             0.001821   
      443           North Dakota             0.018199             0.002006   
      287               Michigan             0.017656             0.000634   
      378          New Hampshire             0.017609             0.001406   
      651                Wyoming             0.017592             0.002425   
      521         South Carolina             0.016734             0.000652   
      326               Missouri             0.016151             0.000633   
      144                 Hawaii             0.015545             0.000864   
      456                   Ohio             0.015445             0.000474   
      1                  Alabama             0.013960             0.000654   
      235              Louisiana             0.013766             0.000621   
      105   District of Columbia             0.012599             0.002997   
      586                Vermont             0.010995             0.001950   
      248                  Maine             0.008099             0.001219   
      313            Mississippi             0.007381             0.000628   
      625          West Virginia             0.

In [8]:
dg = df[['geoname', 'prop_asn_adu_dvap_est', 'prop_asn_adu_dvap_moe']]
dg.sort_values(by=['prop_asn_adu_dvap_est'], ascending=True)

geoname  prop_asn_adu_dvap_est  prop_asn_adu_dvap_moe
table line                                                                    
State 144                 Hawaii               0.146836               0.008220
      365                 Nevada               0.232787               0.010789
      53              California               0.247395               0.002036
      339                Montana               0.290277               0.082064
      261               Maryland               0.292372               0.007687
      118                Florida               0.298106               0.008463
      300              Minnesota               0.301657               0.008924
      599               Virginia               0.302094               0.007543
      105   District of Columbia               0.304099               0.032576
      482                 Oregon               0.317048               0.015059
      66                Colorado               0.321959               0.015032
      14                  Alaska               0.328491               0.028453
      170               Illinois               0.329192               0.005607
      235              Louisiana               0.334145               0.020590
      612             Washington               0.334331               0.006980
      417               New York               0.338007               0.003784
      391             New Jersey               0.339429               0.005264
      404             New Mexico               0.342596               0.037499
      248                  Maine               0.346249               0.043056
      638              Wisconsin               0.349078               0.014168
      521         South Carolina               0.350983               0.019131
      27                 Arizona               0.354527               0.012911
      560                  Texas               0.357139               0.005717
      508           Rhode Island               0.358558               0.033224
      313            Mississippi               0.360845               0.039325
      469               Oklahoma               0.361280               0.018179
      157                  Idaho               0.362711               0.048428
      625          West Virginia               0.365480               0.053676
      495           Pennsylvania               0.371713               0.007076
      274          Massachusetts               0.374508               0.007494
      573                   Utah               0.376240               0.022797
      131                Georgia               0.388217               0.009334
      79             Connecticut               0.390423               0.012376
      430         North Carolina               0.391914               0.009942
      586                Vermont               0.397237               0.060964
      651                Wyoming               0.397546               0.094273
      209                 Kansas               0.400134               0.018985
      1                  Alabama               0.404913               0.020635
      326               Missouri               0.405702               0.014882
      40                Arkansas               0.407611               0.028186
      547              Tennessee               0.409858               0.017156
      287               Michigan               0.416488               0.008453
      222               Kentucky               0.425203               0.020193
      378          New Hampshire               0.425345               0.028698
      664            Puerto Rico               0.436559               0.261840
      92                Delaware               0.440549               0.027733
      456                   Ohio               0.443522               0.012075
      352               Nebraska               0.478135               0.025680
      196                   Iowa               0.487239               0.018

In [9]:
dg = df[['geoname', 'prop_his_adu_dvap_est', 'prop_his_adu_dvap_moe']]
dg.sort_values(by=['prop_his_adu_dvap_est'], ascending=True)

geoname  prop_his_adu_dvap_est  prop_his_adu_dvap_moe
table line                                                                    
State 664            Puerto Rico               0.015439               0.001073
      144                 Hawaii               0.065030               0.008578
      339                Montana               0.081913               0.017860
      248                  Maine               0.092206               0.021653
      14                  Alaska               0.106201               0.016600
      404             New Mexico               0.121973               0.004318
      586                Vermont               0.125212               0.027907
      443           North Dakota               0.129925               0.028519
      625          West Virginia               0.153406               0.040138
      378          New Hampshire               0.176205               0.022281
      495           Pennsylvania               0.181526               0.005709
      651                Wyoming               0.185411               0.020439
      456                   Ohio               0.195153               0.006571
      79             Connecticut               0.217394               0.007169
      66                Colorado               0.222620               0.005676
      287               Michigan               0.229857               0.007175
      27                 Arizona               0.232769               0.004109
      274          Massachusetts               0.235093               0.006075
      508           Rhode Island               0.242740               0.014114
      534           South Dakota               0.246271               0.027909
      118                Florida               0.262462               0.003849
      417               New York               0.262751               0.003539
      326               Missouri               0.266677               0.010536
      560                  Texas               0.275664               0.002426
      157                  Idaho               0.278153               0.011158
      53              California               0.286652               0.002312
      391             New Jersey               0.287437               0.005074
      638              Wisconsin               0.289069               0.007690
      170               Illinois               0.311396               0.004164
      196                   Iowa               0.318313               0.011461
      183                Indiana               0.319931               0.008307
      365                 Nevada               0.328757               0.005928
      209                 Kansas               0.329939               0.008524
      482                 Oregon               0.335960               0.009619
      92                Delaware               0.336143               0.016914
      612             Washington               0.346082               0.006541
      352               Nebraska               0.351077               0.012296
      573                   Utah               0.351688               0.009468
      105   District of Columbia               0.359871               0.021745
      313            Mississippi               0.362028               0.027248
      235              Louisiana               0.362459               0.013320
      469               Oklahoma               0.363865               0.007614
      300              Minnesota               0.377386               0.010833
      599               Virginia               0.382245               0.009263
      222               Kentucky               0.421621               0.014165
      1                  Alabama               0.426478               0.015752
      521         South Carolina               0.428614               0.013191
      40                Arkansas               0.430018               0.012958
      261               Maryland               0.445272               0.009

In [10]:
df[df['geoname'] == 'California'][[f'{r}_adu_est' for r in RACES]]

,,asn_adu_est,blk_adu_est,his_adu_est,ind_adu_est,pac_adu_est,wht_adu_est,tmr_adu_est
table,line,,,,,,,
State,53,4646285,1739065,10643825,113650,112790,12331415,674325


In [11]:
df[['geoname', 'prop_adu_dvap_est']]

geoname  prop_adu_dvap_est
table line                                         
State 1                  Alabama           0.025080
      14                  Alaska           0.039227
      27                 Arizona           0.091322
      40                Arkansas           0.038240
      53              California           0.157527
      66                Colorado           0.065635
      79             Connecticut           0.079764
      92                Delaware           0.057440
      105   District of Columbia           0.087015
      118                Florida           0.103519
      131                Georgia           0.068625
      144                 Hawaii           0.088652
      157                  Idaho           0.044097
      170               Illinois           0.083929
      183                Indiana           0.037317
      196                   Iowa           0.037202
      209                 Kansas           0.052212
      222               Kentucky           0.026348
      235              Louisiana           0.028509
      248                  Maine           0.016921
      261               Maryland           0.084714
      274          Massachusetts           0.088836
      287               Michigan           0.037432
      300              Minnesota           0.046363
      313            Mississippi           0.016995
      326               Missouri           0.025306
      339                Montana           0.011827
      352               Nebraska           0.051640
      365                 Nevada           0.118887
      378          New Hampshire           0.030055
      391             New Jersey           0.113461
      404             New Mexico           0.067142
      417               New York           0.112544
      430         North Carolina           0.056887
      443           North Dakota           0.028392
      456                   Ohio           0.025357
      469               Oklahoma           0.046579
      482                 Oregon           0.063717
      495           Pennsylvania           0.036471
      508           Rhode Island           0.067027
      521         South Carolina           0.033930
      534           South Dakota           0.025680
      547              Tennessee           0.035608
      560                  Texas           0.131012
      573                   Utah           0.066368
      586                Vermont           0.023323
      599               Virginia           0.068072
      612             Washington           0.087259
      625          West Virginia           0.009397
      638              Wisconsin           0.031802
      651                Wyoming           0.025413
      664            Puerto Rico           0.016359

In [12]:
# compare actual disenfranchised adult population to that seen by the Sen
df[~df['geoname'].isin(['District of Columbia', 'Puerto Rico'])]['prop_adu_dvap_est'].mean()

0.05618126459745103

In [13]:
us = load_cvap_data('Nation')
1 - us['cvap_est'] / us['adu_est']

table   line
Nation  1       0.080824
dtype: float64